# Import Libraries

In [1]:
import pandas as pd, numpy as np
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Load Dataset

In [2]:
dataset = pd.read_csv('/content/gdrive/MyDrive/DigitalSkola/Dataset/keywords.csv')
dataset.head()

,Date,Market,Keyword,Average.Position,Clicks,Impressions,Cost
0,20120524,US-Market,secure online back up,0.0,0.00,0.0,0.00
1,20120524,US-Market,agile management software,1.0,21.22,260.0,25.45
2,20120524,US-Market,crm for financial,0.0,0.00,0.0,0.00
3,20120524,US-Market,disaster recovery planning for it,0.0,0.00,0.0,0.00
4,20120524,US-Market,tracking a vehicle,0.0,0.00,0.0,0.00


# Feature Engineering

In [8]:
# CPI Cost per Impressions
dataset['CPI'] = dataset['Cost'] / dataset['Impressions']

# CPC Cost per Click
dataset['CPC'] = dataset['Cost'] / dataset['Clicks']

# CTR Click Through Rate
dataset['CTR'] = dataset['Clicks'] / dataset['Impressions']

# Cost per CTR
dataset['Cost CTR'] = dataset['Cost'] / (dataset['Clicks'] / dataset['Impressions'])


# Data Cleansing
dataset.fillna(value=0, inplace=True)

dataset.head()

,Date,Market,Keyword,Average.Position,Clicks,Impressions,Cost,CPI,CPC,CTR,Cost CTR
0,20120524,US-Market,secure online back up,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000,0.000000
1,20120524,US-Market,agile management software,1.0,21.22,260.0,25.45,0.097885,1.19934,0.081615,311.828464
2,20120524,US-Market,crm for financial,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000,0.000000
3,20120524,US-Market,disaster recovery planning for it,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000,0.000000
4,20120524,US-Market,tracking a vehicle,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000,0.000000


In [11]:
dataset.describe()

,Date,Average.Position,Clicks,Impressions,Cost,CPI,CPC,CTR,Cost CTR
count,3.560540e+05,356054.000000,356054.000000,3.560540e+05,3.560540e+05,356054.000000,356054.000000,356054.000000,3.560540e+05
mean,2.012269e+07,0.606521,435.308986,1.858382e+04,3.567577e+03,0.269194,13.366022,0.016221,1.875281e+05
std,3.577676e+03,0.643551,7527.342527,2.958183e+05,5.693965e+04,0.644417,34.423529,0.020368,2.960094e+06
min,2.012052e+07,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00
25%,2.012090e+07,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00
50%,2.012102e+07,1.000000,1.970000,1.210000e+02,7.230000e+00,0.016697,0.938515,0.007246,3.415033e+02
75%,2.012122e+07,1.000000,34.490000,1.540000e+03,3.648100e+02,0.288975,11.639344,0.027966,1.732963e+04
max,2.013021e+07,12.000000,441963.500000,1.566667e+07,3.269108e+06,33.932771,865.000000,0.239375,2.698246e+08


In [10]:
# Change infinite value to 0
dataset = dataset.replace([np.inf, -np.inf], 0)

# Advanced Label Encoder

In [12]:
dataset.Market.value_counts()

US-Market    231886
UK-Market    124168
Name: Market, dtype: int64

In [14]:
market_data = dataset.pivot_table(index='Market',
                    values=['Clicks', 'Impressions', 'Cost', 'CTR', 'CPC', 'CPI'],
                    aggfunc=[np.mean, np.median, np.max])

In [18]:
mean_market   = market_data['mean'].reset_index()
median_market = market_data['median'].reset_index()
max_market    = market_data['amax'].reset_index()

In [20]:
# change column name
mean_market.columns = ['Market', 'mean_CPC', 'mean_CPI', 'mean_CTR', 'mean_Clicks', 'mean_Cost', 'mean_Impressions']
median_market.columns = ['Market', 'median_CPC', 'median_CPI', 'median_CTR', 'median_Clicks', 'median_Cost', 'median_Impressions']
max_market.columns = ['Market', 'amax_CPC', 'amax_CPI', 'amax_CTR', 'amax_Clicks', 'amax_Cost', 'amax_Impressions']

In [21]:
# Join tables
market_data = pd.merge(mean_market, median_market, how='inner', on='Market')
market_data = pd.merge(market_data, max_market, how='inner', on='Market')

In [22]:
del mean_market, median_market, max_market

In [24]:
market_data

,Market,mean_CPC,mean_CPI,mean_CTR,mean_Clicks,mean_Cost,mean_Impressions,median_CPC,median_CPI,median_CTR,median_Clicks,median_Cost,median_Impressions,amax_CPC,amax_CPI,amax_CTR,amax_Clicks,amax_Cost,amax_Impressions
0,UK-Market,11.971659,0.261776,0.016228,100.861678,1312.905896,3391.964202,0.453896,0.007108,0.004375,0.25,0.52,20.0,756.5,29.065157,0.239375,156341.05,918862.86,9571421.0
1,US-Market,14.112662,0.273165,0.016217,614.395490,4774.885676,26718.610192,1.227160,0.019216,0.008491,5.10,19.99,275.0,865.0,33.932771,0.171930,441963.50,3269107.70,15666669.0


In [25]:
dataset = pd.merge(dataset, market_data, how='left', on='Market')
dataset.head()

,Date,Market,Keyword,Average.Position,Clicks,Impressions,Cost,CPI,CPC,CTR,Cost CTR,mean_CPC,mean_CPI,mean_CTR,mean_Clicks,mean_Cost,mean_Impressions,median_CPC,median_CPI,median_CTR,median_Clicks,median_Cost,median_Impressions,amax_CPC,amax_CPI,amax_CTR,amax_Clicks,amax_Cost,amax_Impressions
0,20120524,US-Market,secure online back up,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000,0.000000,14.112662,0.273165,0.016217,614.39549,4774.885676,26718.610192,1.22716,0.019216,0.008491,5.1,19.99,275.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0
1,20120524,US-Market,agile management software,1.0,21.22,260.0,25.45,0.097885,1.19934,0.081615,311.828464,14.112662,0.273165,0.016217,614.39549,4774.885676,26718.610192,1.22716,0.019216,0.008491,5.1,19.99,275.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0
2,20120524,US-Market,crm for financial,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000,0.000000,14.112662,0.273165,0.016217,614.39549,4774.885676,26718.610192,1.22716,0.019216,0.008491,5.1,19.99,275.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0
3,20120524,US-Market,disaster recovery planning for it,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000,0.000000,14.112662,0.273165,0.016217,614.39549,4774.885676,26718.610192,1.22716,0.019216,0.008491,5.1,19.99,275.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0
4,20120524,US-Market,tracking a vehicle,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000,0.000000,14.112662,0.273165,0.016217,614.39549,4774.885676,26718.610192,1.22716,0.019216,0.008491,5.1,19.99,275.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0


# Handling Text Data

In [26]:
# Install Stop words
!pip install stop-words

  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=95e282a05897547c766e784c4f7034fbd3406a5d458741bf6050230e89d69a9e
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [27]:
import nltk, re
from stop_words import get_stop_words

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [30]:
# Define Stop Words
stop_words = get_stop_words('english')

In [31]:
# Define Tokenization Function
def text_to_token(dataset, column):

    # Tokenize a sentence (change sentence into separated words)
    def my_tokenizer(s):
        s = s.lower()
        tokens = nltk.tokenize.word_tokenize(s)
        tokens = [t for t in tokens if len(t) > 2]
        tokens = [t for t in tokens if re.findall('[\x00-\x7FâäèéêëîïôœùûüÿçÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ]', t)]
        tokens = [t for t in tokens if t not in stop_words]
        return tokens

    # Prepare list to receive tokens
    tokenized = []

    # Count token appearance
    word_index_map = {}

    # Iterate tokenization to all of data
    current_index = 0
    for row in dataset[column]:
        tokens = my_tokenizer(row)
        tokenized.append(tokens)
        for token in tokens:
            if token not in word_index_map:
                word_index_map[token] = current_index
                current_index += 1

    # Transform tokens into vector
    def tokens_to_vector(tokens):
        x = np.zeros(len(word_index_map) + 1)
        for t in tokens:
            i = word_index_map[t]
            x[i] += 1
        x = x / x.sum()
        return x

    # Create features from tokens
    N = len(dataset)
    features = np.zeros((N, len(word_index_map) + 1))
    i = 0
    for tokens in tokenized:
        x = tokens_to_vector(tokens)
        features[i,:] = x
        i += 1
    
    return pd.DataFrame(features)

In [32]:
df = text_to_token(dataset[['Keyword']], 'Keyword')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in true_divide


In [33]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793
0,0.333333,0.333333,0.333333,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.333333,0.333333,0.333333,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.5,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.333333,0.333333,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# SMOTE

In [34]:
dataset.head()

,Date,Market,Keyword,Average.Position,Clicks,Impressions,Cost,CPI,CPC,CTR,Cost CTR,mean_CPC,mean_CPI,mean_CTR,mean_Clicks,mean_Cost,mean_Impressions,median_CPC,median_CPI,median_CTR,median_Clicks,median_Cost,median_Impressions,amax_CPC,amax_CPI,amax_CTR,amax_Clicks,amax_Cost,amax_Impressions
0,20120524,US-Market,secure online back up,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000,0.000000,14.112662,0.273165,0.016217,614.39549,4774.885676,26718.610192,1.22716,0.019216,0.008491,5.1,19.99,275.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0
1,20120524,US-Market,agile management software,1.0,21.22,260.0,25.45,0.097885,1.19934,0.081615,311.828464,14.112662,0.273165,0.016217,614.39549,4774.885676,26718.610192,1.22716,0.019216,0.008491,5.1,19.99,275.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0
2,20120524,US-Market,crm for financial,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000,0.000000,14.112662,0.273165,0.016217,614.39549,4774.885676,26718.610192,1.22716,0.019216,0.008491,5.1,19.99,275.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0
3,20120524,US-Market,disaster recovery planning for it,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000,0.000000,14.112662,0.273165,0.016217,614.39549,4774.885676,26718.610192,1.22716,0.019216,0.008491,5.1,19.99,275.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0
4,20120524,US-Market,tracking a vehicle,0.0,0.00,0.0,0.00,0.000000,0.00000,0.000000,0.000000,14.112662,0.273165,0.016217,614.39549,4774.885676,26718.610192,1.22716,0.019216,0.008491,5.1,19.99,275.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0


In [35]:
dataset.loc[dataset['CTR'] < 0.02, 'target'] = 0
dataset.loc[dataset['CTR'] >= 0.02, 'target'] = 1

dataset['target'].value_counts()

0.0    230137
1.0    125917
Name: target, dtype: int64

In [36]:
from imblearn.over_sampling import SMOTE

In [39]:
X = dataset.drop(['Date', 'Market', 'Keyword'], axis=1)
y = dataset['target']

In [43]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

In [45]:
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [47]:
X_train = pd.DataFrame(X_train, columns=X.columns)
X_train.head()

,Average.Position,Clicks,Impressions,Cost,CPI,CPC,CTR,Cost CTR,mean_CPC,mean_CPI,mean_CTR,mean_Clicks,mean_Cost,mean_Impressions,median_CPC,median_CPI,median_CTR,median_Clicks,median_Cost,median_Impressions,amax_CPC,amax_CPI,amax_CTR,amax_Clicks,amax_Cost,amax_Impressions,target
0,0.0,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000e+00,14.112662,0.273165,0.016217,614.39549,4774.885676,26718.610192,1.22716,0.019216,0.008491,5.1,19.99,275.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0,0.0
1,1.7,0.46,74934.0,108.29,0.001445,235.413043,0.000006,1.764044e+07,14.112662,0.273165,0.016217,614.39549,4774.885676,26718.610192,1.22716,0.019216,0.008491,5.1,19.99,275.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0,0.0
2,1.0,499.94,44082.0,33428.51,0.758326,66.865044,0.011341,2.947545e+06,14.112662,0.273165,0.016217,614.39549,4774.885676,26718.610192,1.22716,0.019216,0.008491,5.1,19.99,275.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0,0.0
3,0.0,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000e+00,14.112662,0.273165,0.016217,614.39549,4774.885676,26718.610192,1.22716,0.019216,0.008491,5.1,19.99,275.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0,0.0
4,1.0,525.78,9589.0,3917.64,0.408556,7.451101,0.054832,7.144861e+04,14.112662,0.273165,0.016217,614.39549,4774.885676,26718.610192,1.22716,0.019216,0.008491,5.1,19.99,275.0,865.0,33.932771,0.17193,441963.5,3269107.7,15666669.0,1.0


In [48]:
X_train['target'].value_counts()

1.0    184145
0.0    184145
Name: target, dtype: int64

In [49]:
X_test['target'].value_counts()

0.0    45992
1.0    25219
Name: target, dtype: int64